# Utils

In [ ]:
# export
# default_exp utils

In [ ]:
# export
def all_equal(iterator):
    " Check if all elements of a list are equal (https://stackoverflow.com/a/3844832/12995344)"
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == x for x in iterator)

In [ ]:
# export
def first_layer(module): 
    " Extract the first layer of a (nested) nn.Sequential"
    while True: 
        try: module = next(module.children())
        except: return module

In [ ]:
# export
def hasattrs(x, attrs, do_raise=False):
    " Check if `x` has all atributes. Optionally raise `ValueError` if some are missing"
    present = [hasattr(x, attr) for attr in attrs]
    all_present = all(present)
    if do_raise and not all_present: 
        attrs = [attr for attr, p in zip(attrs, present) if not p]
        raise ValueError(f'{x.__class__.__name__} has no attributes {attrs}')
    return all_present

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted blocks.ipynb.
Converted index.ipynb.
Converted unet.ipynb.
Converted utils.ipynb.
